In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/balanced_reviews.csv')

In [ ]:
df.shape

(792000, 3)

In [ ]:
df.isnull().sum()

overall         0
reviewText    560
summary       231
dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
df.shape

(791219, 3)

In [ ]:
df.head()

,overall,reviewText,summary
0,5,Great nephew loved!!! Very fast delivery!!!!!,Five Stars
1,5,"well made boot, fits perfectly",Five Stars
2,5,Perfect fit. Good quality. (Do not put in dryer.),Nice bra!
3,5,I bought these for my 2 year old girl and she ...,Better than I expected!
4,5,Great backpack. I wanted to get a backpack th...,Great backpack.


In [ ]:
df.columns.tolist()

['overall', 'reviewText', 'summary']

In [ ]:
x=df['reviewText']
x.shape

(791219,)

In [ ]:
df=df[df['overall']!=3]

In [ ]:
x=df['reviewText']
x.shape

(527383,)

In [ ]:
import numpy as np
df['Positivity'] = np.where(df['overall'] > 3 , 1 , 0)


In [ ]:
y=df['Positivity']
y.shape

(527383,)

In [ ]:
df['Positivity'].value_counts()

0    263815
1    263568
Name: Positivity, dtype: int64

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.8.2'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size=5000

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
messages=x.copy()
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, df.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', df.iloc[i,1])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[1]

'well made boot fit perfectli'

In [ ]:
corpus[0]

'great nephew love fast deliveri'

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4823 1736  482 ...    0    0    0]
 [2916  996  167 ...    0    0    0]
 [1380 4005 4787 ...    0    0    0]
 ...
 [ 217 4179    0 ...    0    0    0]
 [2943 1556 4437 ... 4277  973    0]
 [1553 4005  705 ... 3111 1199  457]]


In [ ]:
## Creating model
embedding_vector_features=500 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 500)           2500000   
                                                                 
 lstm (LSTM)                 (None, 100)               240400    
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 2,740,501
Trainable params: 2,740,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
5522/5522 [==============================] - 41s 6ms/step - loss: 0.3283 - accuracy: 0.8587 - val_loss: 0.3036 - val_accuracy: 0.8708
Epoch 2/10
5522/5522 [==============================] - 39s 7ms/step - loss: 0.2867 - accuracy: 0.8790 - val_loss: 0.2960 - val_accuracy: 0.8740
Epoch 3/10
5522/5522 [==============================] - 34s 6ms/step - loss: 0.2623 - accuracy: 0.8909 - val_loss: 0.2954 - val_accuracy: 0.8750
Epoch 4/10
5522/5522 [==============================] - 35s 6ms/step - loss: 0.2396 - accuracy: 0.9016 - val_loss: 0.2995 - val_accuracy: 0.8748
Epoch 5/10
5522/5522 [==============================] - 35s 6ms/step - loss: 0.2171 - accuracy: 0.9124 - val_loss: 0.3171 - val_accuracy: 0.8725
Epoch 6/10
5522/5522 [==============================] - 39s 7ms/step - loss: 0.1949 - accuracy: 0.9225 - val_loss: 0.3366 - val_accuracy: 0.8711
Epoch 7/10
5522/5522 [==============================] - 39s 7ms/step - loss: 0.1745 - accuracy: 0.9315 - val_loss: 0.3651 - val_ac

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=500
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[86770,     0],
       [87267,     0]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4985721427052868